In [22]:
import tempfile
import pathlib
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

import pyarrow.compute as pc

In [2]:
base = pathlib.Path(tempfile.mkdtemp(prefix="pyarrow-"))

In [3]:
base

PosixPath('/tmp/pyarrow-4lpn979o')

In [13]:
import os
import polars as pl

In [15]:
def get_size(path, unit='MB'):
    size = os.path.getsize(path)
    if unit == 'MB':
        print(f'Size: {size / (1024 * 1024)} MB')
    elif unit == 'GB':
        print(f'Size: {size / (1024 * 1024 * 1024)} GB')
    else:
        print(f'Size: {size} B')

path = './data/test/playlist_2010to2022.parquet'

get_size(path=path)

Size: 0.2697105407714844 MB


In [34]:
get_size('data/test2/audio_features.csv', 'MB')

Size: 458.2281322479248 MB


In [37]:
pl.read_csv('data/test2/audio_features.csv').head()

isrc,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,updated_on
str,f64,f64,i64,f64,f64,i64,f64,f64,i64,f64,i64,i64,f64,str
"""AD4X65752184""",0.906,0.65,296733,0.35,0.775,3,0.114,-11.777,1,0.0284,110,4,0.336,"""2023-08-24 09:…"
"""AEA0D1991170""",0.00095,0.621,191989,0.639,0.82,7,0.506,-9.258,1,0.0775,140,4,0.346,"""2023-08-24 09:…"
"""AEA0Q2004008""",0.0312,0.692,282904,0.98,0.866,9,0.0969,-9.952,1,0.0881,140,4,0.225,"""2023-08-24 09:…"
"""AEA0Q2004009""",0.000558,0.516,197904,0.924,0.434,5,0.153,-13.564,0,0.22,148,4,0.204,"""2023-08-24 09:…"
"""AEA0Q2004010""",0.00938,0.587,199471,0.91,0.804,0,0.563,-10.552,1,0.0596,136,4,0.321,"""2023-08-24 09:…"


In [40]:
from pyarrow import csv

table = csv.read_csv('data/test2/audio_features.csv')
table

pyarrow.Table
isrc: string
acousticness: double
danceability: double
duration_ms: int64
energy: double
instrumentalness: double
key: int64
liveness: double
loudness: double
mode: int64
speechiness: double
tempo: int64
time_signature: int64
valence: double
updated_on: timestamp[s]
----
isrc: [["AD4X65752184","AEA0D1991170","AEA0Q2004008","AEA0Q2004009","AEA0Q2004010",...,"DEAR41525750","DEAR41525761","DEAR41525763","DEAR41526078","DEAR41526200"],["DEAR41526214","DEAR41526249","DEAR41526295","DEAR41526357","DEAR41526411",...,"DEGD91107503","DEGD91107505","DEGD91107507","DEGD91107508","DEGD91107509"],...,["CH6541664408","CH6542352743","NLRD52200994","ITZB42136724","NLHR22100856",...,"DEAR41870698","US8FK1100058","QZ5FN1825826","GBX6A2200154","GBV8E1549001"],["DECH60912164","NLHR22200278","DEAR42267680","AEA2M2348807","CA5KR1510187",...,"DEY472375687","DEFR71400907","GB7NR1743402","DEZN82304108","DEY471812426"]]
acousticness: [[0.906,0.00095,0.0312,0.000558,0.00938,...,0.0098,0.14,0.341,0.

In [41]:
table['isrc']

[
  [
    "AD4X65752184",
    "AEA0D1991170",
    "AEA0Q2004008",
    "AEA0Q2004009",
    "AEA0Q2004010",
    ...
    "DEAR41525750",
    "DEAR41525761",
    "DEAR41525763",
    "DEAR41526078",
    "DEAR41526200"
  ],
  [
    "DEAR41526214",
    "DEAR41526249",
    "DEAR41526295",
    "DEAR41526357",
    "DEAR41526411",
    ...
    "DEGD91107503",
    "DEGD91107505",
    "DEGD91107507",
    "DEGD91107508",
    "DEGD91107509"
  ],
...,
  [
    "CH6541664408",
    "CH6542352743",
    "NLRD52200994",
    "ITZB42136724",
    "NLHR22100856",
    ...
    "DEAR41870698",
    "US8FK1100058",
    "QZ5FN1825826",
    "GBX6A2200154",
    "GBV8E1549001"
  ],
  [
    "DECH60912164",
    "NLHR22200278",
    "DEAR42267680",
    "AEA2M2348807",
    "CA5KR1510187",
    ...
    "DEY472375687",
    "DEFR71400907",
    "GB7NR1743402",
    "DEZN82304108",
    "DEY471812426"
  ]
]